## Tutorial 8 - Sparse Matrices and Benchmarking
In this tutorial we'll use sparse matrix representations of stuff we've done previously: Hamiltonian and Poisson Matrices, and also learn some tools that we can use to benchmark our code; find out how long things take to operate, or how much file size they consume.

In [99]:
import numpy as np
from matplotlib import pyplot as plt
import time
import scipy.sparse as sparse

In [100]:
#Keeping our Tri-Diagonal Matrix creator from Tutorial 7
def TriD_H(nodes, t, V):
    #nodes is the number of nodes in our 1D system.
    #t is the hbar^2/(2ma^2) component of the Hamiltonian
    #V is a vector of length nodes that contains the potential (voltage) at each node in our potential well
    
    # This is a modified version of the dri-diagonal matrix maker from last time
    assert len(V) == nodes
    
    # Create our empty matrix
    TriD = np.zeros((nodes,nodes))
    for i in range(nodes):
        row = np.zeros(nodes)
        if i == 0:
            # First row
            row[i] = 2*t+V[i]
            row[i+1] = -1*t
        elif i == (nodes-1):
            # End row
            row[i-1] = -1*t
            row[i] = 2*t + V[i]
        else:
            # Middle rows
            row[i-1] = -1*t
            row[i] = 2*t + V[i]
            row[i+1] = -1*t
        
        TriD[i,:] = row
    
    return TriD

First, we're going to experiment with sparse matrices using the scipy.sparse package. It has a number of different ways of 'compressing' a sparse matrix (which we will call 'representations'). It also has some specific operations designed to be efficient on sparse matrices.

In [109]:
# Defining some parameters: the only one that matters here is 'nodes'; the rest is just fill data
nodes = 500
t = 20 
V = np.zeros(nodes)

# Make our matrix and print it to see.
H = TriD_H(nodes,t,V)
print('Uncrompressed Matrix:')
print(H)

# Let's have a look at different sparse representations of this matrix.
H_csc = sparse.csc_matrix(H)
print('\nCompressed Sparse Column:')
print(H_csc)
H_csr = sparse.csr_matrix(H)
print('\nCompressed Sparse Row:')
print(H_csr)
H_lil = sparse.lil_matrix(H)
print('\nList of Lists')
print(H_lil)
H_dok = sparse.dok_matrix(H)
print('\nDictionary of Keys')
print(H_dok)

# There doesn't appear to be much difference *in the print out*, but there may be a lot going on behind the scenes.

# We can take any of our matrices and convert them to a 'regular' matrix format by calling the .todense function
H_normal = H_csr.todense()
# Back to a normal matrix:
print(H_normal)

Uncrompressed Matrix:
[[ 40. -20.   0. ...   0.   0.   0.]
 [-20.  40. -20. ...   0.   0.   0.]
 [  0. -20.  40. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  40. -20.   0.]
 [  0.   0.   0. ... -20.  40. -20.]
 [  0.   0.   0. ...   0. -20.  40.]]

Compressed Sparse Column:
  (0, 0)	40.0
  (1, 0)	-20.0
  (0, 1)	-20.0
  (1, 1)	40.0
  (2, 1)	-20.0
  (1, 2)	-20.0
  (2, 2)	40.0
  (3, 2)	-20.0
  (2, 3)	-20.0
  (3, 3)	40.0
  (4, 3)	-20.0
  (3, 4)	-20.0
  (4, 4)	40.0
  (5, 4)	-20.0
  (4, 5)	-20.0
  (5, 5)	40.0
  (6, 5)	-20.0
  (5, 6)	-20.0
  (6, 6)	40.0
  (7, 6)	-20.0
  (6, 7)	-20.0
  (7, 7)	40.0
  (8, 7)	-20.0
  (7, 8)	-20.0
  (8, 8)	40.0
  :	:
  (491, 491)	40.0
  (492, 491)	-20.0
  (491, 492)	-20.0
  (492, 492)	40.0
  (493, 492)	-20.0
  (492, 493)	-20.0
  (493, 493)	40.0
  (494, 493)	-20.0
  (493, 494)	-20.0
  (494, 494)	40.0
  (495, 494)	-20.0
  (494, 495)	-20.0
  (495, 495)	40.0
  (496, 495)	-20.0
  (495, 496)	-20.0
  (496, 496)	40.0
  (497, 496)	-20.0
  (496, 497)	-20.0
  (497, 497)	40

Where the more interesting stuff comes in to play is when you try to modify / perform an operation with the sparse matrices: you can't really use normal numpy operations! Instead you need to use the spefic operations available as part of the spmatrix class: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spmatrix.html#scipy.sparse.spmatrix

(They are called by putting a .METHOD() after the variable name, for example H_csr.get_shape())

In [110]:
# These are some sparse matrix example functions you can do on sparse matrices.
t_start = time.perf_counter()
print("Calculating the mean value:")
print("Regular matrix representation")
print(np.mean(H))
print('\nCompressed Sparse Column:')
print(H_csc.mean())
print('\nCompressed Sparse Row:')
print(H_csr.mean())
print('\nList of Lists')
print(H_lil.mean())
print('\nDictionary of Keys')
print(H_dok.mean())

Calculating the mean value:
Regular matrix representation
0.00016

Compressed Sparse Column:
0.00015999999999999999

Compressed Sparse Row:
0.00015999999999999999

List of Lists
0.00016

Dictionary of Keys
0.00016


Let's now look at some performance metrics. There are two 'figures of merit' that we care about: how much time it takes to do something with the matrix, how much space a representation takes up. Time we are going to analyse using the time.perf_counter() function, and space we are going to the VAR_NAME.data.nbytes() command)

In [116]:
#timing something is pretty simple; you get a start time, a finish time, and subtract them from each other.
start = time.perf_counter()
for i in range(100000000):
    a = 1+2
end = time.perf_counter()

print('Time taken (s):')
print(end-start)

Time taken (s):
4.487857000000076


In [125]:
# Let's time how long it takes to make a sparse representation of a matrix:
nodes = 10000
t = 20 
V = np.zeros(nodes)
H = TriD_H(nodes,t,V)

print('Making a LIL sparse Matrix of size N = %s'% nodes)
start = time.perf_counter()
H_lil = sparse.lil_matrix(H)
end = time.perf_counter()
print('Time taken (s):')
print(end-start)

print('Making a CSR sparse Matrix of size N = %s'% nodes)
start = time.perf_counter()
H_csr = sparse.csr_matrix(H)
end = time.perf_counter()
print('Time taken (s):')
print(end-start)

Making a LIL sparse Matrix of size N = 10000
Time taken (s):
0.5743222999999489
Making a CSR sparse Matrix of size N = 10000
Time taken (s):
0.5565918000002057


In [122]:
# Let's see how large our representations are:
nodes = 10000
t = 20
V = np.zeros(nodes)
H = TriD_H(nodes,t,V)

print('Size of matrix (in MB):')
print(H.data.nbytes/(1024**2))

print('\nSize of LIL representation (in MB):')
H_lil = sparse.lil_matrix(H)
print(H_lil.data.nbytes/(1024**2))

print('\nSize of CSR representation (in MB):')
H_csr = sparse.csr_matrix(H)
print(H_csr.data.nbytes/(1024**2))

Size of matrix (in MB):
762.939453125

Size of LIL representation (in MB):
0.0762939453125

Size of CSR representation (in MB):
0.2288665771484375


**Beginner Tasks**
1. Create your own matrix (maybe 10x10) in a separate cell, and find the sparse representations of it. Try different shapes (i.e. lots of off diagonal terms, everything concentrated in one corner) and have a look at how the representations change.
2. Time how long it takes to sparse-ify a matrix of size 10,000 by 10,000 using the time.perf_counter() tool. Which method (CSR, CSC, LIL, DOK etc) is the quickest?
3. Repeat the above timings but for de-sparsing a matrix (using the matrix.todense() command). Which representation has the overall best time? Does it change depending on what type of matrix you use (i.e. heavily diagonal, completely full, identity matrix etc)?

**Intermediate Tasks**
1. Investigate the scaling properties of one sparse representation of your choosing. As a function of initial matrix size (N), determine:

    a. How strongly the time-to-compress and time-to-unpack change.
    
    b. How much the file size of the un-compressed and compressed matrix change.
    
    
2. For your chosen sparse representation, compare the timings of the following methods for calculating the dot product of two matrices:

    a. use np.dot() in the dense (i.e. regular) matrix representation
    
    b. compress both matrices to sparse matrices, and use the spmatrix.dot() representation, then uncompress.

**Advanced Tasks**
1. Repeat the intermediate tasks (rip) but change the values in the matrices to be of double precision. What are the differences?
1. Repeat the intermetiate timing tasks **again** but time multiple runs of the same method; is there much variability in how long certain operations take?